In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

In [2]:
with open('../Data/2023_edges_header.csv', 'r') as file:
    columns = file.readline().strip().split(',') 
df_edges = pd.read_csv('../Data/2023_edges.csv', header=None)
df_edges.columns = columns
df_edges

,:START_ID,starttimems:int,endtimems:int,:END_ID,start_date:DATETIME,:TYPE
0,0,1698266696468,1698267092113,509,2023-10-25T20:44:56.468000,CYCLES_TO
1,1,1698270535502,1698271967951,301,2023-10-25T21:48:55.502000,CYCLES_TO
2,2,1698285705974,1698285992418,471,2023-10-26T02:01:45.974000,CYCLES_TO
3,3,1698269489932,1698269814383,424,2023-10-25T21:31:29.932000,CYCLES_TO
4,4,1698270006571,1698270440298,1641,2023-10-25T21:40:06.571000,CYCLES_TO
...,...,...,...,...,...,...
11730057,412,1685820631664,1685823750407,532,2023-06-03T19:30:31.664000,CYCLES_TO
11730058,492,1685820631467,1685821009127,3346,2023-06-03T19:30:31.467000,CYCLES_TO
11730059,910,1685820628918,1685822020021,311,2023-06-03T19:30:28.918000,CYCLES_TO
11730060,1268,1685820635166,1685822157916,1789,2023-06-03T19:30:35.166000,CYCLES_TO


In [3]:
#from "weatherstats.ca based on Environment and Climate Change Canada data"
df_weather = pd.read_csv('../Data/2023_forecast_hourly.csv')
df_weather['period_string'] = pd.to_datetime(df_weather['period_string'])
df_weather_2023 = df_weather[df_weather['period_string'].dt.year == 2023]
df_weather_2023.reset_index(drop=True, inplace=True)
df_weather_2023 = df_weather_2023[['period_string','conditions','temperature','wind_speed']]
df_weather_2023.drop_duplicates(subset=['period_string'], keep='first', inplace=True)
df_weather_2023

,period_string,conditions,temperature,wind_speed
0,2023-12-31 16:00:00,Possibilité d'averses de neige,-5,15
1,2023-12-31 17:00:00,Possibilité d'averses de neige,-5,15
2,2023-12-31 18:00:00,Possibilité d'averses de neige,-5,15
3,2023-12-31 19:00:00,Possibilité d'averses de neige,-5,15
4,2023-12-31 20:00:00,Possibilité d'averses de neige,-6,15
...,...,...,...,...
33940,2023-01-01 00:00:00,Pluie,4,10
33941,2023-01-01 01:00:00,Pluie,4,10
33942,2023-01-01 02:00:00,Pluie,4,10
33943,2023-01-01 03:00:00,Pluie,3,10


In [4]:
df_edges['start_date:DATETIME'] = pd.to_datetime(df_edges['start_date:DATETIME'])

# Convert 'period_string' column in df_weather_2023 to datetime format
df_weather_2023['period_string'] = pd.to_datetime(df_weather_2023['period_string'])
# Extract year, month, day, and hour from 'start_date:DATETIME' column in df_edges
df_edges['year'] = df_edges['start_date:DATETIME'].dt.year
df_edges['month'] = df_edges['start_date:DATETIME'].dt.month
df_edges['day'] = df_edges['start_date:DATETIME'].dt.day
df_edges['hour'] = df_edges['start_date:DATETIME'].dt.hour

# Merge df_edges and df_weather_2023 based on year, month, day, and hour
df_edges_weather = pd.merge(df_edges, df_weather_2023, 
                            left_on=['year', 'month', 'day', 'hour'], 
                            right_on=[df_weather_2023['period_string'].dt.year,
                                      df_weather_2023['period_string'].dt.month,
                                      df_weather_2023['period_string'].dt.day,
                                      df_weather_2023['period_string'].dt.hour], 
                            how='left')
df_edges_weather

,:START_ID,starttimems:int,endtimems:int,:END_ID,start_date:DATETIME,:TYPE,year,month,day,hour,period_string,conditions,temperature,wind_speed
0,0,1698266696468,1698267092113,509,2023-10-25 20:44:56.468,CYCLES_TO,2023,10,25,20,2023-10-25 20:00:00,Généralement nuageux,18.0,30.0
1,1,1698270535502,1698271967951,301,2023-10-25 21:48:55.502,CYCLES_TO,2023,10,25,21,2023-10-25 21:00:00,Généralement nuageux,18.0,15.0
2,2,1698285705974,1698285992418,471,2023-10-26 02:01:45.974,CYCLES_TO,2023,10,26,2,2023-10-26 02:00:00,Généralement nuageux,16.0,15.0
3,3,1698269489932,1698269814383,424,2023-10-25 21:31:29.932,CYCLES_TO,2023,10,25,21,2023-10-25 21:00:00,Généralement nuageux,18.0,15.0
4,4,1698270006571,1698270440298,1641,2023-10-25 21:40:06.571,CYCLES_TO,2023,10,25,21,2023-10-25 21:00:00,Généralement nuageux,18.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11730057,412,1685820631664,1685823750407,532,2023-06-03 19:30:31.664,CYCLES_TO,2023,6,3,19,2023-06-03 19:00:00,Ensoleillé,18.0,20.0
11730058,492,1685820631467,1685821009127,3346,2023-06-03 19:30:31.467,CYCLES_TO,2023,6,3,19,2023-06-03 19:00:00,Ensoleillé,18.0,20.0
11730059,910,1685820628918,1685822020021,311,2023-06-03 19:30:28.918,CYCLES_TO,2023,6,3,19,2023-06-03 19:00:00,Ensoleillé,18.0,20.0
11730060,1268,1685820635166,1685822157916,1789,2023-06-03 19:30:35.166,CYCLES_TO,2023,6,3,19,2023-06-03 19:00:00,Ensoleillé,18.0,20.0


In [5]:
import math

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [6]:
with open('../Data/2023_nodes_header.csv', 'r') as file:
    columns = file.readline().strip().split(',') 
df_nodes = pd.read_csv('../Data/2023_nodes.csv', header=None)
df_nodes.columns = columns
df_nodes
distances = []
for idx1, station1 in df_nodes.iterrows():
    for idx2, station2 in df_nodes.iterrows():
        if idx1 != idx2:
            lon1, lat1 = station1['longitude:float'], station1['latitude:float']
            lon2, lat2 = station2['longitude:float'], station2['latitude:float']
            dist = haversine(lat1,lon1,lat2,lon2)
            distances.append((station1['stationId:ID'],station2['stationId:ID'],dist))

In [7]:
for i in range(min(5, len(distances))):
    print(distances[i])

(0, 1, 7.215101983863039)
(0, 2, 2.907664170953294)
(0, 3, 0.3775092829394819)
(0, 4, 1.8956378410606136)
(0, 5, 4.828137009253774)


In [8]:
# Create a dictionary to map node pairs to their distances
node_pairs_to_distance = {}

# Populate the dictionary with distances
for distance in distances:
    node1, node2, dist = distance
    # Add distance for both directions of the edge
    node_pairs_to_distance[(node1, node2)] = dist
    node_pairs_to_distance[(node2, node1)] = dist

# Create a new column 'distance' in the edges DataFrame
df_edges_weather['distance'] = [
    node_pairs_to_distance.get((start_node_id, end_node_id), None)
    for start_node_id, end_node_id in zip(df_edges_weather[':START_ID'], df_edges_weather[':END_ID'])
]

# Now df_edges_weather contains a new column 'distance' with the distances for each edge
df_edges_weather = df_edges_weather.drop([':TYPE','year','period_string'],axis=1)
df_edges_weather

,:START_ID,starttimems:int,endtimems:int,:END_ID,start_date:DATETIME,month,day,hour,conditions,temperature,wind_speed,distance
0,0,1698266696468,1698267092113,509,2023-10-25 20:44:56.468,10,25,20,Généralement nuageux,18.0,30.0,1.265884
1,1,1698270535502,1698271967951,301,2023-10-25 21:48:55.502,10,25,21,Généralement nuageux,18.0,15.0,4.621719
2,2,1698285705974,1698285992418,471,2023-10-26 02:01:45.974,10,26,2,Généralement nuageux,16.0,15.0,1.216274
3,3,1698269489932,1698269814383,424,2023-10-25 21:31:29.932,10,25,21,Généralement nuageux,18.0,15.0,0.838028
4,4,1698270006571,1698270440298,1641,2023-10-25 21:40:06.571,10,25,21,Généralement nuageux,18.0,15.0,1.127610
...,...,...,...,...,...,...,...,...,...,...,...,...
11730057,412,1685820631664,1685823750407,532,2023-06-03 19:30:31.664,6,3,19,Ensoleillé,18.0,20.0,3.358278
11730058,492,1685820631467,1685821009127,3346,2023-06-03 19:30:31.467,6,3,19,Ensoleillé,18.0,20.0,1.206038
11730059,910,1685820628918,1685822020021,311,2023-06-03 19:30:28.918,6,3,19,Ensoleillé,18.0,20.0,3.490008
11730060,1268,1685820635166,1685822157916,1789,2023-06-03 19:30:35.166,6,3,19,Ensoleillé,18.0,20.0,3.364704


In [9]:
# Save column names to a CSV file
column_names = df_edges_weather.columns

with open('../Data/2023_edges_header.csv', 'w') as f:
    f.write(','.join(column_names))

df_edges_weather.to_csv('../Data/2023_edges.csv',index=False, header=False)
